HSE, Applied Time Series Forecasitng , Winter 2023-24

<font color="green"> Lesson #13: Bonus Topics </font>

<span style="color:black; font-size: 12pt"></span>

Alexey Romanenko,
<font color="blue">alexromsput@gmail.com</font>

**Key words:**
 demand restoration, value from forecasting

**Your feedback:**  please provide you feedback  <a href="https://forms.gle/bEV5iwmTqdM9rUoy8"> here </a>

In [18]:
import numpy as np
import pandas as pd
from copy import deepcopy

# don't forget to upload utils file to the colab session
import warnings, pylab
from statsmodels.tsa.holtwinters import SimpleExpSmoothing
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.metrics import mean_absolute_error, mean_squared_error

warnings.filterwarnings('ignore')


import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
pd.options.plotting.backend = "plotly"

# Using TS forecasting packages. Online Demo

## Online Demo based on HW3

In [5]:
# загружаем данные
ts_hw3 = pd.read_csv('train.csv', sep=',')
ts_hw3['period_start_dt'] = pd.to_datetime(ts_hw3['period_start_dt'], format= "%Y-%m-%d") #dayfirst=True)
ts_hw3.set_index(['period_start_dt'], inplace=True)
ts_hw3 = ts_hw3.sort_index() # must have
ts_hw3.head()

,Unnamed: 0,product_rk,store_location_rk,demand,PROMO1_FLAG,PROMO2_FLAG,PRICE_REGULAR,PRICE_AFTER_DISC,NUM_CONSULTANT,AUTORIZATION_FLAG
period_start_dt,,,,,,,,,,
2016-12-19,0,40369,309,29.0,NaN,NaN,NaN,NaN,NaN,NaN
2016-12-19,21361,40369,1034,20.0,NaN,NaN,NaN,NaN,NaN,NaN
2016-12-19,21362,40370,1034,31.0,NaN,NaN,NaN,NaN,NaN,NaN
2016-12-19,21363,40372,1034,11.0,NaN,NaN,NaN,NaN,NaN,NaN
2016-12-19,21364,40373,1034,14.0,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
# transform data to original format
df_actual = ts_hw3.reset_index().rename(columns = {'demand':'metricValue', 'period_start_dt':'date'})
df_actual['metric']= 'demand'
df_actual['status']= 'actual'

# df_actual[['date', 'metric', 'metricValue', 'product_rk', 'store_location_rk', 'status']]

# конвертим дату в datetime и сортируем данные по дате
df_actual['date'] = pd.to_datetime(df_actual['date'])
df_actual = df_actual.sort_values('date')

# убираем тестовые данные
df_actual = df_actual[df_actual.date<'2019-12-01']

# определяем поля, в которых зашит id одного ts
groupby_columns = ['product_rk', 'store_location_rk']
# конвертируем поля id-ts в string
df_actual[groupby_columns] = df_actual[groupby_columns].astype(str)

In [40]:
# find largest ts (in terms of sum of target variable)
_list_ts = df_actual.groupby(groupby_columns)['metricValue'].mean().sort_values(ascending = False)
_list_ts[:5]

,,metricValue
product_rk,store_location_rk,
40370,309,105.000000
40369,309,68.666667
40370,1281,64.000604
40372,309,50.000000
40370,425,42.622383


In [160]:
# extract 5
df_actual[df_actual['product_rk']== '40370'].set_index('date')['2019-12-01':]


ts_hw3['2019-09-01':'2019-12-01'].groupby(groupby_columns).mean()['demand'].sort_values(ascending = False)

product_rk  store_location_rk
40370       1281                 31.923077
            1328                 23.000000
40372       1281                 22.461538
96212       862                  21.705128
40369       1347                 21.615385
                                   ...    
96212       533                   0.000000
            540                   0.000000
40369       1202                  0.000000
40373       1080                  0.000000
            533                   0.000000
Name: demand, Length: 240, dtype: float64

In [128]:
# рисуем графиики каждого из ts, чтобы увидеть скрытые компоненты ts (тренд и сезонность)
# можно делать stl - декомпозицию, но как правило все можно увидеть на простом графике

# analysis of top-5 ts
# df_actual = _list_ts[:5]


def plot_by_ts(df, groupby_columns, target_col = 'metricValue', date_col_name = 'date', granularity='D', aggregation_method = 'sum'):
    # from plotly.subplots import make_subplots
    pd.options.plotting.backend = "plotly"

    plot_df = df.set_index(date_col_name).groupby(groupby_columns).resample(granularity).agg({target_col:aggregation_method}).reset_index().set_index([date_col_name] + groupby_columns)[target_col]
    # plot_df = df.set_index([date_col_name] + groupby_columns)[[target_col]]

    # транспонируем все group_by колонки:
    for i in range(len(groupby_columns)):
        plot_df = plot_df.unstack()
    # print(plot_df.head())

    # делаем название колонок "плоскими"
    plot_df.columns = [' '.join(col).strip() for col in plot_df.columns.values]

    # рисуем графики
    plot_df.plot().show()
    return

plot_by_ts(df_actual[df_actual['product_rk'] == '40370'], groupby_columns, granularity = 'W-MON')

### Вывод:
### Сезонность наблюдается: скорее во всех ts
### Тренд наблюдается Черновики заявки - Казань, Черновики заявки - Уфа

### предлается брать additivную компонету никаких ярких трендов не видно,  (пердлагается взять простое ES или для учета плохо заметно недельной несезонности сезонную модель ES)

In [92]:
pd.date_range(start=df_actual.date.max() + pd.Timedelta(days=1), periods=5, freq='W-MON')
# ts_hw3.index.unique()

DatetimeIndex(['2019-12-02', '2019-12-09', '2019-12-16', '2019-12-23',
               '2019-12-30'],
              dtype='datetime64[ns]', freq='W-MON')

In [149]:
# функция для запуска прогноза по всем ts
def ets_forecast(ts, steps=30, date_column = 'date', target_column = 'metricValue', granularity = 'D', **kwargs):

    # ts.set_index(pd.DatetimeIndex(ts[date_column]).to_period(granularity))[target_column].plot().show()

    model = ExponentialSmoothing(ts.set_index(pd.DatetimeIndex(ts[date_column]).to_period(granularity))[target_column], **kwargs).fit()

    forecast = model.forecast(steps)
    return forecast

def run_forecast(df, forecast_horizon, params, groupby_columns, granularity = 'D'):
    #иницируем df, куда складываем прогноз
    df_forecast = df[:0]

    # по каждому ts отдельно
    for ts_id in params.keys(): # df.groupby(groupby_columns).count().index:

        idx_ = (df[groupby_columns]==ts_id).all(axis=1)

        # стром прогноз
        ts_forecast = pd.DataFrame({
        'date': pd.date_range(start=df[idx_].date.max() + pd.Timedelta(days=1), periods=forecast_horizon, freq=granularity),
        'metricValue': eval("ets_forecast(df[idx_][['metricValue', 'date']],steps=forecast_horizon,"+params[ts_id]+", granularity = granularity)")
            })

        # заполняем поля groupby columns
        for i in range(len(groupby_columns)):
            ts_forecast[groupby_columns[i]] = [ts_id[i]]*forecast_days

        # добавляем прогноз ts в общий df прогнозов
        df_forecast = pd.concat([df_forecast,ts_forecast], ignore_index=True)

    return df_forecast

In [69]:
# default model for ts
ids = df_actual['product_rk'] == '40370'
df_actual = df_actual[ids]
{ts_id: "trend = None, seasonal = None" for ts_id in df_actual.groupby(groupby_columns).count().index.to_list()}

{('40370', '1005'): 'trend = None, seasonal = None',
 ('40370', '1034'): 'trend = None, seasonal = None',
 ('40370', '1079'): 'trend = None, seasonal = None',
 ('40370', '1080'): 'trend = None, seasonal = None',
 ('40370', '1143'): 'trend = None, seasonal = None',
 ('40370', '1162'): 'trend = None, seasonal = None',
 ('40370', '1173'): 'trend = None, seasonal = None',
 ('40370', '1185'): 'trend = None, seasonal = None',
 ('40370', '1191'): 'trend = None, seasonal = None',
 ('40370', '1202'): 'trend = None, seasonal = None',
 ('40370', '1203'): 'trend = None, seasonal = None',
 ('40370', '1281'): 'trend = None, seasonal = None',
 ('40370', '1316'): 'trend = None, seasonal = None',
 ('40370', '1326'): 'trend = None, seasonal = None',
 ('40370', '1328'): 'trend = None, seasonal = None',
 ('40370', '1347'): 'trend = None, seasonal = None',
 ('40370', '1363'): 'trend = None, seasonal = None',
 ('40370', '1380'): 'trend = None, seasonal = None',
 ('40370', '309'): 'trend = None, seasonal = N

In [165]:
# задаем для каджого ts конкретную модель (указываем тренд/сезонность/границы параметров сг)
# df_actual.groupby(groupby_columns).count().index[[0,1,2,3,4,5]].to_list()
# рекомендация для выбора сезонности: подневные данные - 7, понедельные - 52 (53), помесячные - 12, поквартальные данные - 4

forecast_weeks = 5

# каждый ts добавляем только 1 раз
params = {
#     ('40370', '1005'): "seasonal = 'add', seasonal_periods =52, trend='add'",
#  ('40370', '1034'): "seasonal = 'add', seasonal_periods =52, trend='add'",
 ('40370', '1079'): "seasonal = 'add', seasonal_periods =53, trend='add', bounds = bounds3",
#  ('40370', '1080'): "seasonal = 'add', seasonal_periods =52, trend='add'",
#  ('40370', '1143'): "seasonal = 'add', seasonal_periods =52, trend='add'",
#  ('40370', '1162'): "seasonal = 'add', seasonal_periods =52, trend='add'",
#  ('40370', '1173'): "seasonal = 'add', seasonal_periods =52, trend='add'",
#  ('40370', '1185'): "seasonal = 'add', seasonal_periods =52, trend='add'",
#  ('40370', '1191'): "seasonal = 'add', seasonal_periods =52, trend='add'",
#  ('40370', '1202'): "seasonal = 'add', seasonal_periods =52, trend='add'",
#  ('40370', '1203'): "seasonal = 'add', seasonal_periods =52, trend='add'",
#  ('40370', '1281'): "seasonal = 'add', seasonal_periods =53, trend='add'",
 ('40370', '1328'): "seasonal = 'add', seasonal_periods =53, trend='add', bounds = bounds3"
 }


# bounds позволяет настроить уровень/тренда/сезонность:
# чем больше хотите сделать похожим на последние даты, тем больше (ближе к 1) должны быть границы
# ВАЖНО: большие значения параметров сглаживания зачастую эквиваленты переобучению модели ets (== не перестарайтесь сделать прогноз похожим на последние даты)
bounds1 = {'smoothing_level':(0.0, 0.01), 'smoothing_seasonal':(0.3,0.6)} #  , 'initial_seasonal.0':(9,10). 'smoothing_trend':(0.9, 0.999)
bounds3 = {'smoothing_level':(0.0, 0.1), 'smoothing_seasonal':(0.9,0.999)} #  'smoothing_trend':(0.9, 0.999)
bounds2 = {'smoothing_level':(0.0, 0.01)} #  , 'initial_seasonal.0':(9,10). 'smoothing_trend':(0.9, 0.999)
bounds4 = {'smoothing_level':(0.0, 0.01), 'smoothing_seasonal':(0.7,0.8)} #  'smoothing_trend':(0.9, 0.999)

# запускаем построение проноза по всем ts
df_forecast = run_forecast(df=df_actual, forecast_horizon = forecast_weeks, params = params, groupby_columns = groupby_columns, granularity = 'W-MON')

# собираем прогноз и факт в один датафрейм
df_forecast['status'] = 'baseline'
df_actual['status'] = 'fact'
df_combined = pd.concat([df_actual, df_forecast], ignore_index=True)
# df_combined.tail()


## Валидируем результат

In [166]:
# Разделение данных на обучающую и тестовую выборки
def split_data(df, test_size=38):
    train, test = df.set_index('date')[:-test_size], df.set_index('date')[-test_size:]
    return train.reset_index(), test.reset_index()

# Рассчитываем метрики
def calculate_metrics(true_values, predictions):
    mae = mean_absolute_error(true_values, predictions)
    mse = mean_squared_error(true_values, predictions)
    rmse = np.sqrt(mse)
    # print(predictions + true_values)
    wape = np.sum(np.abs((true_values - predictions))) # / np.sum(np.abs((true_values))) * 100
    return mae, mse, rmse, wape

def compare_models(df, models_array, groupby_columns, date_column = 'date', target_column = 'metricValue', granularity = 'D', draw = False, **kwargs):

    # создаем df для складывания ошибки прогноза
    score_df = df.groupby(groupby_columns).count().reset_index()[groupby_columns][:0]
    score_df['score_metric'] = ''
    score_df['score_value'] = np.NaN
    score_df['model'] = ''

    # для каждого варианта моделей
    for m_params in models_array:
        # по каждому ts
        for ts_id in m_params.keys(): # df_actual.groupby(groupby_columns).count().index:
            # print(ts_id)
            idx_ = (df[groupby_columns]==ts_id).all(axis=1)

            train, test = split_data(df[idx_][[target_column, date_column]])
            train[[target_column, date_column]].set_index(date_column).plot().show()
            # test.set_index(date_column).plot().show()
            ts_fcst = pd.DataFrame({
                                date_column: test.index,
                                target_column: eval("ets_forecast(train,steps=test.shape[0],"+m_params[ts_id]+", granularity = granularity)")
                                }).set_index(date_column)

            mae, mse, rmse, wape, = calculate_metrics(test[target_column], ts_fcst[target_column])

            # сохраняем ошибки в df
            score_df.loc[len(score_df.index)] = list(ts_id) + list(['mae', mae]) + list([m_params[ts_id]])
            score_df.loc[len(score_df.index)] = list(ts_id) + list(['mse', mse]) + list([m_params[ts_id]])
            score_df.loc[len(score_df.index)] = list(ts_id) + list(['rmse', rmse]) + list([m_params[ts_id]])
            score_df.loc[len(score_df.index)] = list(ts_id) + list(['wape', wape]) + list([m_params[ts_id]])
            # if True:
            #   ts_fcst[[target_column]].merge(test, left_index = True, right_index = True).set_index(date_column).plot().show()

    return score_df

# альтератинвый вариант: все ts прогнозируем с помощью Simple ES
params2 = {ts_id: "trend = None, seasonal = None" for ts_id in params.keys()}

score_df = compare_models(df_actual,[params, params2], groupby_columns, granularity = 'W-MON')

ValueError: Cannot compute initial seasonals using heuristic method with less than two full seasonal cycles in the data.

In [154]:
# cравневаем модели по точности прогнозирования на валидацинном периоде
# ошибка итоговой модели НЕобязательно лучшая, но не должна быть значительно выше (==хуже), чем у других
score_df.set_index(groupby_columns + ['model', 'score_metric']).unstack('score_metric') #.reset_index()

score_value  \
score_metric                                                                            mae   
product_rk store_location_rk model                                                            
40370      1079              seasonal = 'add', seasonal_periods =53, trend='...   11.662165   
                             trend = None, seasonal = None                        14.081411   
           1326              seasonal = 'add', seasonal_periods =53, trend='...   23.469129   
                             trend = None, seasonal = None                        55.307806   

                                                                                              \
score_metric                                                                             mse   
product_rk store_location_rk model                                                             
40370      1079              seasonal = 'add', seasonal_periods =53, trend='...  1392.338002   
                             trend = None, seasonal = None                        225.295465   
           1326              seasonal = 'add', seasonal_periods =53, trend='...  6519.815347   
                             trend = None, seasonal = None                       3140.832548   

                                                                                            \
score_metric                                                                          rmse   
product_rk store_location_rk model                                                           
40370      1079              seasonal = 'add', seasonal_periods =53, trend='...  37.314046   
                             trend = None, seasonal = None                       15.009846   
           1326              seasonal = 'add', seasonal_periods =53, trend='...  80.745374   
                             trend = None, seasonal = None                       56.043131   

                                                                                              
score_metric                                                                            wape  
product_rk store_location_rk model                                                            
40370      1079              seasonal = 'add', seasonal_periods =53, trend='...   443.162284  
                             trend = None, seasonal = None                        535.093601  
           1326              seasonal = 'add', seasonal_periods =53, trend='...   891.826916  
                             trend = None, seasonal = None                       2101.696629

In [155]:
# Визуализация прогнозов | Аддитивная сезонность
# если прогноз не подходит - тогда меняем модель (возвращаемся на шаг назад)
# plot_by_ts(df_combined,  groupby_columns=groupby_columns+['status'], granularity= 'D')

plot_by_ts(df_combined,  groupby_columns=groupby_columns+['status'], granularity= 'W-MON')

### метод валидации: смотрим на едкватность сезонности и тренда + смотрим на прогноз в разных гранулярностях

## Python packages for time series forecasting

https://github.com/aromanenko/ATSF/wiki/List-of-packages-for-Time-Series-Forecasting-in-Python

# Business Value from Forecasting
 Business case: your company launched new Demand Forecasting System. What couldn't be as a business-value due to this implementation?
  
  a) free money due less overforecasting or less underforecasting
  
  b) free resources of analytics (less cost of support resources)
  
  c) increase transparancy and consistency in decision making processes (new money will come)
  
  d) speed up decision making process (more money due to fast reaction, more frequent decision making process)